# Machines Manufacturing Capital Budgeting Model

#### The Problem
<p>You work for a new startup that is trying to manufacture phones. You are tasked with building a model which will
help determine how many machines to invest in and how much to spend on marketing.</p> 

<p>Each machine produces n output phones per year. Each phone sells for p phone and costs c phone in variable costs to produce. After n life years, the machine can no longer produce output, but may be scrapped for p scrap.</p> 

<p>The machine will not be replaced, so you may end up with zero total output before your model time period ends. Equity investment is limited, so in each year you can spend cmachine to either buy a machine or buy advertisements. In the first year you must buy a machine.</p>

<p>Any other machine purchases must be made one after another (advertising can only begin after machine buying is done). 
Demand for your phones starts at d1. Each time you advertise, demand increases by gd%. The prevailing market interest rate is r.</p>

#### Notes
- You may limit your model to 20 years and a maximum of 5 machines if it is helpful.
- For simplicity, assume that cmachine is paid in every year, even after all machines have shut down.
- Ensure that you can change the inputs and the outputs change as expected.
- For simplicity, assume that fractional phones can be sold, you do not need to round the quantity transacted.

[**Setup**](#Setup): Runs any imports and other setup<br>
[**Inputs**](#Inputs): Defines the inputs for the model

#### Bonus Problem
<p>It is unrealistic to assume that price and demand are unrelated. To extend the model, we can introduce a relationship between price and demand,
 given by the following equation:</p>

$d_{1}$ = $d_{c}$ − $E_{phone}$ <br>
$E_{phone}$: Price elasticity of demand <br>
$d_{c}$: Demand constant

<p>For elasticities and constants [(E = 500, d c = 900000), (E = 200, d c = 500000), (E = 100, d c = 300000)] (3 total cases), 
 and taking the other model inputs in the Check your Work section, determine the optimal price for each elasticity, that is the price which maximizes the NPV.</p>

#### Notes
- $d_{1}$ is no longer an input, but an output.
- This bonus requires optimization. (In Python, the scipy package provides optimization tools.)

MODEL MADE BY [Thiago Guarino](https://github.com/thiagoguarino)

### SETUP

Setup for the later calculations are here. The necessary packages are imported.

In [1]:
from dataclasses import dataclass
import numpy_financial as npf

### INPUTS

All of the inputs for the model are defined here. A class is constructed to manage the data, and an instance of the class containing the default inputs is created.

In [2]:
@dataclass
class ModelInputs:
    n_phones: float = 100000
    price_scrap: float = 50000
    price_phone: float = 500
    cost_machine_adv: float = 1000000
    cogs_phone: float = 250
    n_life: int = 10
    n_machines: int = 5
    d_1: float = 100000
    g_d: float = 0.2
    max_year: float = 20
    interest: float = 0.05
        
    # Inputs for bonus problem
    elasticity: float = 100
    demand_constant: float = 300000
        
model_data = ModelInputs()
model_data


ModelInputs(n_phones=100000, price_scrap=50000, price_phone=500, cost_machine_adv=1000000, cogs_phone=250, n_life=10, n_machines=5, d_1=100000, g_d=0.2, max_year=20, interest=0.05, elasticity=100, demand_constant=300000)

### MODEL

In [3]:
class Phone_Manufacturing:

    def __init__(self, input: ModelInputs):
        self.input = input
    
    def get_product_demand(self):
        demands = []
        for d in range(1,(self.input.max_year + 1)):   
            if d <= (self.input.n_machines):
                demand = (self.input.d_1)
                demands.append(demand)
            else:
                demand = (self.input.d_1) * (1 + self.input.g_d) ** (d - (self.input.n_machines))
                demands.append(demand)
        return demands
    
    def get_produced_products(self):
        produced_quant = []
        for a in range(1,(self.input.max_year + 1)):
            if a * self.input.n_phones <= (self.input.n_machines * self.input.n_phones):
                prod = self.input.n_phones * a
                produced_quant.append(prod)

            elif a <= self.input.n_life:
                prod = self.input.n_machines * self.input.n_phones
                produced_quant.append(prod)

            elif a <= (self.input.n_machines + self.input.n_life):
                prod = (self.input.n_machines * self.input.n_phones) - (self.input.n_phones) * (a - self.input.n_life)
                produced_quant.append(prod)

            elif a > (self.input.n_life + self.input.n_machines):
                prod = 0
                produced_quant.append(prod)
        return produced_quant

    def get_sales_revenue(self, demands, produced_quant):
        revenues = []
        for r in range(1,(self.input.max_year + 1)):
            if demands[r-1] <= produced_quant[r-1]:
                revenue = round(self.input.price_phone * demands[r-1])
                revenues.append(revenue)
            else:
                revenue = round(self.input.price_phone * produced_quant[r-1])
                revenues.append(revenue)
        return revenues

    def get_cost_sales(self, demands, produced_quant):
        cost_sales = []
        for s in range(1,(self.input.max_year + 1)):
            if demands[s-1] <= produced_quant[s-1]:
                cost = round(self.input.cogs_phone * demands[s-1])
                cost_sales.append(cost)
            else:
                cost = round(self.input.cogs_phone * produced_quant[s-1])
                cost_sales.append(cost)
        return cost_sales

    def get_profits(self, revenues, cost_sales):
        profits = []
        for p in range(1,(self.input.max_year + 1)):
            profit = revenues[p-1] - cost_sales[p-1]
            profits.append(profit)
        return profits
    
    def get_cash_flows_from_profits(self, profits, initial_investment=0):
        '''
        returns a list containing the cash flows
        '''

        cash_flows = [initial_investment]

        for c in range(1,(self.input.max_year+1)):
            if (c > self.input.n_life) and (c <= (self.input.n_life + self.input.n_machines)):
                c_flow = profits[c-1] - (self.input.cost_machine_adv) + (self.input.price_scrap)
                cash_flows.append(c_flow)
            else:
                c_flow = (profits[c-1] - (self.input.cost_machine_adv))
                cash_flows.append(c_flow)
        return cash_flows

    def get_cash_flows(self):
        demands = self.get_product_demand()
        product_products = self.get_produced_products()
        revenues = self.get_sales_revenue(demands, product_products)
        cost_sales = self.get_cost_sales(demands, product_products)
        profits = self.get_profits(revenues, cost_sales)
        return self.get_cash_flows_from_profits(profits, initial_investment=0)


In [4]:
cash_flow_model = Phone_Manufacturing(model_data)
cash_flows = cash_flow_model.get_cash_flows()

print('Cash Flows: ')
for n, cf in enumerate(cash_flows[1:]):
    print(f'Year {n + 1}: {cf}')

#net present value function
npv_c = npf.npv(model_data.interest, cash_flows)
print(f'NPV: {npv_c:.2f}')

Cash Flows: 
Year 1: 24000000
Year 2: 24000000
Year 3: 24000000
Year 4: 24000000
Year 5: 24000000
Year 6: 29000000
Year 7: 35000000
Year 8: 42200000
Year 9: 50840000
Year 10: 61208000
Year 11: 73699600
Year 12: 74050000
Year 13: 49050000
Year 14: 24050000
Year 15: -950000
Year 16: -1000000
Year 17: -1000000
Year 18: -1000000
Year 19: -1000000
Year 20: -1000000
NPV: 369276542.47
